### Dr. Chakra Chennubhotla Lab -
### add details here...

In [2]:
import networkx
import dgl

import torch
import torch.nn as nn
import torch.nn.functional as F

from dgl.data.utils import save_graphs
from dgl.data.utils import load_graphs

import sklearn
import pickle 
from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import imageio
import cv2
import matplotlib as mpl;
from matplotlib import cm
import pickle as pkl
import glob
import os
import math
from scipy.io import loadmat # Added package
from shapely.geometry import Polygon, MultiPolygon, Point # Added package

from scipy.spatial import distance
from collections import defaultdict
from math import sqrt
from scipy.interpolate import splprep, splev, CubicSpline
from sklearn.mixture import GaussianMixture
from collections import defaultdict
from PIL import Image, ImageDraw
from scipy.ndimage.measurements import label
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from networkx import draw

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
from scipy import io
from sklearn import metrics
plt.rcParams.update({'figure.max_open_warning': 0})
import random

import os.path

import sys
sys.path.insert(1,'gnn-min-example-master/')
import torch.utils.data

from core.dataloader.constants import TRAIN_RATIO, TEST_RATIO
from core.utils import read_params
from core.model import Model

from torch.nn.parallel import DistributedDataParallel as DDP
import torch.multiprocessing as mp
import torch.distributed as dist

In [3]:
def get_graph(feats_path, imagename, k_nearest, dmin):
    
    if os.path.isfile(feats_path+imagename+'_mask_measurements.csv') == True:
        nuclei_metrics = pd.read_csv(feats_path+imagename+'_mask_measurements.csv') 
    elif os.path.isfile(feats_path+imagename+'_measurements.csv') == True:
        nuclei_metrics = pd.read_csv(feats_path+imagename+'_measurements.csv') 
        
    fcg = nuclei_metrics.drop([' ', 'Label', '%Area', 'StdDev', 'Min', 'Max', 'Median', 'Skew', 'Kurt', 'X', 'Y'], 1)
    #fcg = nuclei_metrics.drop(['X', 'Y'], 1)
    print(nuclei_metrics.shape[0])
    while(nuclei_metrics.empty != True and nuclei_metrics.shape[0] >=50):
    
        xcoords = np.asarray(nuclei_metrics['X'])
        ycoords = np.asarray(nuclei_metrics['Y'])

        # dmin = 50 pixels
        # k = 5
        # build the graph - CG by connecting atmost 5 nearest nuclei which is no greater than 50 pixels apart

        coords = np.zeros((xcoords.shape[0],2))
        coords[:,0] = xcoords
        coords[:,1] = ycoords

        dist_nuc = distance.squareform(distance.pdist(coords))
        dist_nuc_sorted = np.sort(dist_nuc, axis=1)

        closest = np.argsort(dist_nuc, axis=1)
        k_closest = np.zeros((k_nearest, closest.shape[1]))
        k_closest = closest[:,1:k_nearest+1]

        # get 5 closest distances

        k_distance = dist_nuc_sorted[:,0:k_nearest+1]

        src = []
        dst = []
        
        aa = []
        for i in range(fcg.shape[0]):

            for j in range(k_nearest):

                # check if nucleus if more than 50 pixels away (dmin)

                if k_distance[i,j] <= dmin:

                    src.append(i);
                    dst.append(k_closest[i,j])

        u = np.concatenate([src, dst])
        v = np.concatenate([dst, src])
        u = u.astype(np.int32)
        v = v.astype(np.int32)
        
        G = dgl.graph((u, v))

        cell_features = torch.tensor(fcg.values)
        cell_features = cell_features.float()
        
        G.ndata['feat'] = cell_features
    
        return G
    else:
        return 0

## Binary classification

### Create train, val, and test data ready for dataloader

In [4]:
label_path = '/home/akashparvatikar/Documents/cvpr/data/'

# get image list for train, val, and test
train = pd.read_csv(label_path+'tf_train.csv')
val   = pd.read_csv(label_path+'tf_val.csv')
test  = pd.read_csv(label_path+'tf_test.csv')

#### 1. Load train labels and graphs

In [5]:
train_images = np.array(train['Image'])
train_labels = train['Label']

In [6]:
with open('train_data_invasive', 'rb') as config_dictionary_file:
    train_data = pickle.load(config_dictionary_file)

#### 2. Load validation labels and graphs

In [7]:
val_images = np.array(val['Image'])
val_labels = val['Label']

In [8]:
with open('val_data_invasive', 'rb') as config_dictionary_file:
    val_data = pickle.load(config_dictionary_file)

#### 3. Load test labels and graphs

In [9]:
test_images = np.array(test['Image'])
test_labels = test['Label']

In [10]:
with open('test_data_invasive', 'rb') as config_dictionary_file:
    test_data = pickle.load(config_dictionary_file)

## Run CG-GNN

In [11]:
def make_data_loader(dataset_train,dataset_val,dataset_test, batch_size, cuda, gpu):

    preprocess(dataset_train, cuda)
    preprocess(dataset_val, cuda)
    preprocess(dataset_test, cuda)
    
    rank = args.nr * args.gpus + gpu
    train_sampler = torch.utils.data.distributed.DistributedSampler(dataset_train,num_replicas=args.world_size,rank=rank)
    val_sampler = torch.utils.data.distributed.DistributedSampler(dataset_val,num_replicas=args.world_size,rank=rank)
    test_sampler = torch.utils.data.distributed.DistributedSampler(dataset_test,num_replicas=args.world_size,rank=rank)
    
    train_dataloader = torch.utils.data.DataLoader(dataset_train,
                                                   batch_size=batch_size,
                                                   shuffle=False,
                                                   collate_fn=collate, num_workers=0, pin_memory=True, 
                                                   sampler=train_sampler)

    val_dataloader = torch.utils.data.DataLoader(dataset_val,
                                                 batch_size=batch_size, 
                                                 shuffle=False,
                                                 collate_fn=collate, num_workers=0, pin_memory=True, 
                                                 sampler=val_sampler)

    test_dataloader = torch.utils.data.DataLoader(dataset_test,
                                                  batch_size=batch_size, 
                                                  shuffle=False,
                                                  collate_fn=collate, num_workers=0, pin_memory=True, 
                                                  sampler=test_sampler)
                                                  

    return train_dataloader, val_dataloader, test_dataloader

In [12]:
def collate(data):
    """
    Collate function
    """
    graphs, labels = map(list, zip(*data))
    batched = dgl.batch(graphs)
    labels = torch.LongTensor(labels)
    return batched, labels


def preprocess(dataset, cuda):

    for g, _ in dataset:
        g,_ = g.to('cuda:0'), _.to('cuda:0')
        for key_g, val_g in g.ndata.items():
            processed = g.ndata.pop(key_g)
            processed = processed.type('torch.FloatTensor')
            if cuda:
                processed = processed.cuda()
            g.ndata[key_g] = processed
        for key_g, val_g in g.edata.items():
            processed = g.edata.pop(key_g)
            processed = processed.type('torch.FloatTensor')
            if cuda:
                processed = processed.cuda()
            g.edata[key_g] = processed

## Run GNN main function

In [13]:
#!/usr/bin/env python3
"""
Run model script.
"""
import torch
import argparse
import numpy as np
from core.utils import read_params
from core.model import Model


def main(gpu, args):
    
    ############################################################
    rank = args.nr * args.gpus + gpu                         
    dist.init_process_group(backend='nccl', init_method='env://', world_size=args.world_size, rank=rank)
    ############################################################
    
    config_params = read_params(args.config_fpath)
    
    if args.gpu < 0:
        cuda = False
    else:
        cuda = True
        torch.cuda.set_device(args.gpu)
        
    SAVE_DIR = 'models'
    MODEL_SAVE_PATH = os.path.join(SAVE_DIR, 'mlp-cgnn_v2_invasive.pt')
    
    if not os.path.isdir(f'{SAVE_DIR}'):
        os.makedirs(f'{SAVE_DIR}')
            
    best_valid_loss = float('inf')
    
    print('*** Create data loader ***')
    dataloader, val_dataloader, test_dataloader = make_data_loader(train_data, val_data
                                                               ,test_data, batch_size=16, cuda=True, gpu=gpu)
    
    print('*** Create model ***')
    model = Model(config=config_params, verbose=True, cuda=cuda)
    
    print(cuda)
    if cuda==True:
        torch.cuda.set_device(args.gpu)
        
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=args.lr,
                                 weight_decay=args.weight_decay)
    # loss function
    loss_fcn = torch.nn.CrossEntropyLoss()
    
    # Start training
    print('*** Start training ***')
    step = 0
    model.train()
    
    # DDP
    model = model.to(args.gpu)
    model = nn.parallel.DistributedDataParallel(model.to(args.gpu), device_ids=[args.gpu])
    
    losses = []
    for epoch in range(args.n_epochs):
        print("Epoch:", epoch)
        for iter, (graphs, labels) in enumerate(dataloader):
            
            # forward pass
            graphs = graphs.to('cuda')
            logits = model(graphs)
            labels = labels.to('cuda')
            # compute loss
            loss = loss_fcn(logits, labels)
            losses.append(loss.item())

            # backpropagate
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # testing
            step += 1
            if step % args.eval_every == 0:
                val_loss, val_acc = test(val_dataloader, model, loss_fcn)
                print(
                    "Step {:05d} | Train loss {:.4f} | Over {} | Val loss {:.4f} |"
                    "Val acc {:.4f}".format(
                        step,
                        np.mean(losses),
                        len(losses),
                        val_loss,
                        val_acc,
                    ))
                model.train()
                
                if val_loss < best_valid_loss:
                    best_valid_loss = val_loss
                    torch.save(model.state_dict(), MODEL_SAVE_PATH)

def test(data_loader, model, loss_fcn):
    """
    Testing
    :param data_loader: (data.Dataloader)
    :param model: (Model)
    :param loss_fcn: (torch.nn loss)
    :return: loss, accuracy
    """
    model.eval()
    losses = []
    accuracies = []
    
    true_labels = []
    pred_labels = []
    for iter, (graphs, labels) in enumerate(data_loader):

        graphs = graphs.to('cuda')
        logits = model(graphs)
        labels = labels.to('cuda')
        loss = loss_fcn(logits, labels)
        losses.append(loss.item())

        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        accuracies.append(correct.item() * 1.0 / len(labels))

    return np.mean(losses), np.mean(accuracies)

In [15]:
if __name__ == '__main__':
    
    class Args:
        
        lr = 1e-3
        weight_decay = 5e-4
        n_epochs = 10
        batch_size = 16
        eval_every = 50
        config_fpath = 'gnn-min-example-master/core/config/config_file_binary.json'
        gpu=0
        
        # DDP
        gpus = 1
        nodes = 1
        nr = 0
        world_size = gpus * nodes
    
    
    args = Args()
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '8802' 
    mp.spawn(main(0,args), nprocs= gpus, args = Args())
    
    #main(args)

*** Create data loader ***
*** Create model ***
Creating GNN layers...
Creating new GNN layer:
MLP layer 0 has params Sequential(
  (fc): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
MLP layer 1 has params Sequential(
  (fc): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)
Creating new GNN layer:
MLP layer 0 has params Sequential(
  (fc): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)
MLP layer 1 has params Sequential(
  (fc): Linear(in_features=14, out_features=14, bias=True)
  (bn): BatchNorm1d(14, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
)
Creating new GNN layer:
MLP layer 0 has params Sequential(
  (fc): Linear(in_features=14, out_features=14, bias

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.




Epoch: 0
Epoch: 1
Step 00050 | Train loss 0.5179 | Over 50 | Val loss 0.5764 |Val acc 0.7250
Epoch: 2
Step 00100 | Train loss 0.5039 | Over 100 | Val loss 0.5395 |Val acc 0.7188
Epoch: 3
Step 00150 | Train loss 0.4949 | Over 150 | Val loss 0.5285 |Val acc 0.7688
Epoch: 4
Epoch: 5
Step 00200 | Train loss 0.4857 | Over 200 | Val loss 0.5930 |Val acc 0.6000
Epoch: 6
Step 00250 | Train loss 0.4790 | Over 250 | Val loss 0.5317 |Val acc 0.7438
Epoch: 7


KeyboardInterrupt: 

In [ ]:
# destroy WORLD process if required

In [16]:
torch.distributed.destroy_process_group()